# Main script to clean UW final satellite pm2.5 data

## Model 1: county_ssn

Modules: N/A <br>
Author: Cornelia Ilin <br>
Email: cilin@wisc.edu <br>
Date created: Oct 20, 2020 <br>

**Citations (code and data)**
1. https://zenodo.org/record/2616769#.X5CTHtBKg4d
2. https://sites.wustl.edu/junmeng/research/historical-pm2-5-estimation/

    
**Citations (persons)**
1. N/A

**Preferred environment**
1. Code written in Jupyter Notebooks

#### Step 1: Import packages

In [1]:
# standard
import pandas as pd
import numpy as np
import os
import h5py


In [2]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install osmnx

Note: you may need to restart the kernel to use updated packages.


In [4]:
# geography
import geopandas as gpd
import osmnx as ox

#### Step 2: Define working directories

In [5]:
in_dir_cnty = "/Users/katiecason/Desktop/207_folder/w207_finalproject/"
in_dir = "/Users/katiecason/Desktop/207_folder/w207_finalproject/"

In [6]:
os.chdir(in_dir)

#### Step 3: Define functions

``read data``

In [9]:
def read_osmnx_geom():
    """ Read osmnx (lat, lon) coordinates for California counties
    parameters:
    -----------
    None
    
    return:
    -------
    Df with osmnx_geom
    """
    ### Step 1 ### 
    ##############
    # Grab the name and code for each county in California
    for file in os.listdir(in_dir_cnty):
        if file.startswith('data_selection'):
            cnty = pd.read_excel(os.path.join(in_dir_cnty, file),'County_names', skiprows=1)

    # keep only cols of interest
    cnty = cnty[['county_code', 'county_name']]
    
    ### Step 2 ###
    ###############
    # For each county extract osmnx polygon with (lat, lon) info
    osmnx_geom = pd.DataFrame()
    for idx, cnty_name in enumerate(cnty.county_name.values):
        # extract multipoly counties
        if cnty_name in ["San Francisco", "Santa Barbara", "Ventura", "Napa"]:
            for multipoly in ox.geocode_to_gdf(cnty_name + ', California, USA').geometry:
                #print(multipoly)
                for poly in multipoly:
                    temp_xy = poly.exterior.coords.xy
                    temp_xy_df = pd.DataFrame({'latitude': temp_xy[1], 'longitude': temp_xy[0], 'county_name': cnty_name})
                    osmnx_geom = pd.concat([osmnx_geom, temp_xy_df], axis=0)
        # extract poly counties
        if cnty_name in ['Glenn', 'Imperial']:
            for poly in ox.geocode_to_gdf(cnty_name + ', California, USA').geometry:     
                # extract lat, lon info
                temp_xy = poly.exterior.coords.xy
                # transform to df
                temp_xy_df = pd.DataFrame({'latitude': temp_xy[1], 'longitude': temp_xy[0], 'county_name': cnty_name})      
                osmnx_geom = pd.concat([osmnx_geom, temp_xy_df], axis=0)

    # round (lat, lon) to 2 decimal points and add 0.005 to match the UW (lat, lon) values
    osmnx_geom['latitude'] = osmnx_geom.latitude.round(2) + 0.005
    osmnx_geom['longitude'] = osmnx_geom.longitude.round(2) + 0.005
    osmnx_geom.sort_values(by=['latitude', 'longitude'], inplace=True)
    osmnx_geom.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)

    # Add county code to osmnx_geom
    osmnx_geom = osmnx_geom.merge(cnty, on='county_name', how='left')
    
    return osmnx_geom

In [10]:
def read_uw_pm25(osmnx_geom):
    """Read UW pm25 data
    parameters:
    -----------
    osmnx_geom: df, contains osmnx_geom and county name/code
    
    return:
    df with pm25 values by year and county in California
    """
    df = pd.DataFrame()
    
    for idx, file in enumerate(os.listdir(in_dir)):
        if file.endswith('.h5'):
            # read data
            f = h5py.File(os.path.join(in_dir, file), 'r')
            # read latitude
            row_index = f['latitude']
            row_index = pd.DataFrame(row_index, columns=['latitude'])
            # read longitude
            col_index = f['longitude']
            col_index = pd.DataFrame(col_index, columns = ['longitude'])
            # read pm25 (divide by 100 as indicated here: https://zenodo.org/record/2616769#.X4999NBKg4c)
            pm25 = f['CorrectedPM2.5']
            pm25 = pd.DataFrame(pm25)/100

            # add col and row index to pm25_df
            pm25.set_index(row_index.latitude.values, inplace=True)
            pm25.columns = col_index.longitude.values
            pm25.reset_index(drop=False, inplace=True)
            pm25.rename(columns={'index':'latitude'}, inplace=True)

            # melt pm25_df
            pm25 = pd.melt(pm25, id_vars='latitude', var_name='longitude', value_vars=col_index.longitude.values, value_name='pm25')
            pm25.sort_values(by=['latitude', 'longitude'], inplace=True)

            # set lat and lon to 3 decimals
            pm25['latitude'] = pm25.latitude.round(3)
            pm25['longitude'] = pm25.longitude.astype(float).round(3)

            # add year column
            pm25['year'] = file[:4]

            # merge with osmnx_geom
            pm25 = osmnx_geom.merge(pm25, on=['latitude', 'longitude'], how='inner')

            # group by county and year
            pm25 = pm25.groupby(['county_name', 'county_code', 'year'], as_index=False).agg({'pm25': np.mean})

            # add year and cnty column
            pm25['year_cnty'] = pm25.year.astype(str) + '.0_' + pm25.county_code.astype(str) + '.0'

            # append to df
            df = pd.concat([df, pm25], axis=0)

    return df

#### Step 4: Read data

In [11]:
osmnx_geom = read_osmnx_geom()

UnboundLocalError: local variable 'cnty' referenced before assignment

In [ ]:
df = read_uw_pm25(osmnx_geom)

In [8]:
df.head()

,county_name,county_code,year,pm25,year_cnty
0,Glenn,11,2016,4.676581,2016.0_11.0
1,Imperial,13,2016,7.730566,2016.0_13.0
2,Napa,28,2016,6.742143,2016.0_28.0
3,San Francisco,38,2016,4.018800,2016.0_38.0
4,Santa Barbara,42,2016,8.534000,2016.0_42.0


In [9]:
df.sort_values(by=['county_name', 'year'], inplace=True)

In [10]:
df.to_csv(in_dir + 'UW_pm25.csv')